In [48]:
import os
import json
import cv2
import string
import random
import albumentations as A
import copy

In [79]:
template_directory = 'templates_img'

text_locations = json.load(open('text_locations.json', 'r'))

backgrounds_dir = 'image_backgrounds'

In [18]:
# {'license_driver', 'dob', 'address_full', 'zip', 'name_last', 'email', 'name', 'name_first', 'business', 'state', 'initial_middle','address', 'city', 'relation'}

def generate_field(field):
    if field == 'ssn':
        res = ''.join(random.choices(string.digits, k=9))
        return res
    elif field == 'tele':
        res = ''.join(random.choices(string.digits, k=10))
        return res


def generate_random_field(length = 0):
    length = length if length else random.randint(2, 20)
    field = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=length))
    return field

transform = A.Compose([
    # A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.005),
    A.RandomBrightnessContrast(p=0.3),
    A.ImageCompression(quality_lower=10,quality_upper=100,p=.5),
    A.AdvancedBlur(p=.4),
    A.InvertImg(p=.001),
    A.ISONoise(p=0.5),
    # A.HueSaturationValue(p=0.1),
    A.GaussNoise(p=0.5),
    A.RandomSunFlare(p=0.2)
])

generate_random_field()

'>BzhCn>+7>'

In [63]:
transform = A.Compose([
        # A.RandomRotate90(),
        # A.Flip(),
        # A.Transpose(),
        A.ImageCompression(quality_lower=50, p=0.5),
        A.OneOf([
            # A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(p=0.8),
            A.ISONoise(p=0.2),
            A.MultiplicativeNoise(p=.05)
        ], p=0.5),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=30, p=0.3),
        A.OneOf([
            A.OpticalDistortion(p=0.5),
            A.GridDistortion(p=.5),
            A.PiecewiseAffine(p=0.5),
        ], p=0.4),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.OneOf([
            A.RandomFog(),
            A.RandomRain(),
            A.RandomSnow(),
            A.RandomSunFlare(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
        A.ToGray(always_apply=True)
    ])
# random.seed(42) 

In [64]:
fonts = [
    cv2.FONT_HERSHEY_SIMPLEX,
    cv2.FONT_HERSHEY_COMPLEX,
    cv2.FONT_HERSHEY_PLAIN,
    cv2.FONT_HERSHEY_DUPLEX,
    cv2.FONT_HERSHEY_TRIPLEX,
    cv2.FONT_HERSHEY_COMPLEX_SMALL,
    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,
    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,
    cv2.FONT_ITALIC]

In [40]:

for filename in text_locations:
    if text_locations[filename] != {}:
        img = cv2.imread(template_directory+ '/'+filename, 1)
        img = cv2.resize(img, (850, 1100)) 

        doc_info = text_locations[filename]
        for loc in doc_info:
            font = random.choice(fonts)
            cv2.putText(img, generate_random_field(),
                        (int(loc['x']),int(loc['y'])), font,
					    0.75, (255, 0, 0), 2)

        transformed = transform(image=img)
        img = transformed['image']
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [69]:
backgrounds = []
for filename in os.listdir(backgrounds_dir):
    backgrounds.append(cv2.imread(backgrounds_dir+ '/'+filename, 1))

for filename in text_locations:
    if text_locations[filename] != {}:
        image = cv2.imread(template_directory+ '/'+filename, 1)
        image = cv2.resize(image, (850, 1100)) 

        doc_info = text_locations[filename]
        for i in range(5):
            large_img = random.choices(backgrounds)
            img = image
            for loc in doc_info:
                font = random.choice(fonts)
                cv2.putText(img, generate_random_field(),
                            (int(loc['x']),int(loc['y'])), font,
                            0.75, (255, 0, 0), 2)
            # resize(210,275)
            x_size = random.randint(-200,400)
            y_size = random.randint(-200,400)
            x_size = x_size if x_size > 0 else 0
            y_size = y_size if y_size > 0 else 0
            x_offset = int(x_size/2)
            y_offset = int(y_size/2)
            background_img = cv2.resize(background_img, (850+x_size, 1100+y_size)) 

            background_img[y_offset:y_offset+img.shape[0], x_offset:x_offset+img.shape[1]] = img
            transformed = transform(image=cv2.resize(img, (210,275)))
            img = transformed['image']
            # img = cv2.cvtColor(transformed['image'], cv2.COLOR_BGR2GRAY)
            # img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
            cv2.imwrite(f'test_data/{filename}_{i}', img)
            print(i,end='\r')


In [78]:
backgrounds_dir = 'image_backgrounds'

for filename in os.listdir(backgrounds_dir):
    background_img = cv2.imread(backgrounds_dir+ '/'+filename, 1)
    x_size = random.randint(-200,200)
    y_size = random.randint(-200,200)
    x_size = x_size if x_size > 0 else 0
    y_size = y_size if y_size > 0 else 0
    x_offset = int(x_size/2)
    y_offset = int(y_size/2)
    background_img = cv2.resize(background_img, (850+x_size, 1100+y_size)) 

    out = background_img
    out[y_offset:y_offset+img.shape[0], x_offset:x_offset+img.shape[1]] = img
    cv2.imshow('image', background_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img

In [75]:
0/2

0.0